<a href="https://colab.research.google.com/github/lhoegler16/Biersortenerkennung/blob/main/Userinterface_Biersortenerkenner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Classifier zu Biersorten
##Folgende Biersorten werden erkannt:

* Baumgartner
  * Baumgartner Export
  * Baumgartner Märzen
* Goesser
  * Goesser Bock
  * Goesser Märzen
  * Gösser Naturgold
* Guinnes
  * Guinness Draught
  * Guinness Dublin Porter
  * Guinness Golden ALe
  * Guinness Nitro IPA
  * Guinness Special Export
  * Guinness West Indies Porter
* Rieder
  * Rieder Märzen
  * Rieder Pils
  * Rieder Urecht
  * Rieder Weisse
* Stiegl
  * Stiegl Bio Zwickl
  * Stiegl Goldbräu
  * Stiegl Stieglbock
  * Stiegl Weisse
* Wieselburger
  * Wieselburger Gold
  * Wieselburger Schwarzbier
  * Wieselburger Stammbraeu
  * Wiselburger Zwickl
* Zipfer
  * Zipfer Drei
  * Zipfer Märzen
  * Zipfer Stefanibock
  * Zipfer Urtyp



In [4]:
!pip install fastai --upgrade -q

In [9]:
from fastai.vision.all import*
import ipywidgets as widgets
import requests

In [15]:
def download_file_from_google_drive(id, destination):
    URL="https://docs.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(URL, params = {'id' : id},stream = True)
    token = get_confirm_token(response)
    if token:
        params = {'id' : id  , 'confirm' : token}
        response = session.get(URL,params= params, stream = True)
    save_response_content(response,destination)
def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None
def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [16]:
file_id = "1-10QV8OJx9taXBuyI61AOwGrhywPTpn1"
destination = './biersorten_classifier.pkl'
download_file_from_google_drive(file_id,destination)

path = Path();
inference = load_learner(path/'biersorten_classifier.pkl', cpu=True)

In [17]:
btn_upload = widgets.FileUpload()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')
out_p1 = widgets.Output()
def on_click_classify(change):
  img = PILImage.create(btn_upload.data[-1])
  out_p1.clear_output()
  with out_p1: display(img.to_thumb(128,128))
  pred,pred_idx,probs = inference.predict(img)
  lbl_pred.value = f'Prediction: {pred} ; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)
widgets.VBox([widgets.Label('Select your beer'),
      btn_upload, btn_run,out_p1,lbl_pred])